# Vietnamese ASR (ZipFormer-30M RNNT) — Colab

This notebook runs the Vietnamese ZipFormer-30M RNNT model from this repo and exposes a public HTTP API for testing. It auto‑bootstraps when opened directly from GitHub on Colab.


In [ ]:
#@title 1) Bootstrap + Environment setup (ffmpeg, deps, ngrok)
import os, sys, subprocess, textwrap, pathlib, shutil

REPO_URL = 'https://github.com/nguyenlee97/auto-speech-regconition-vietnamese-deployment.git'
TARGET_DIR = '/content/auto-speech-regconition-vietnamese-deployment'

def sh(cmd: str, check: bool = True):
    print(cmd)
    return subprocess.run(cmd, shell=True, check=check)

# If opened from GitHub, clone and cd into repo so files are present
if not (os.path.exists('requirements.txt') and os.path.exists('model.py')):
    if os.path.exists(TARGET_DIR):
        os.chdir(TARGET_DIR)
    else:
        sh(f'git clone {REPO_URL} {TARGET_DIR}')
        os.chdir(TARGET_DIR)
print('CWD:', os.getcwd())

# ffmpeg for format conversion
sh('apt-get update -y')
sh('apt-get install -y --no-install-recommends ffmpeg')

# Python deps
print('Python:', sys.version)
sh('python -m pip install --upgrade pip wheel setuptools')
py_minor = sys.version_info.minor
if py_minor <= 10 and os.path.exists('requirements.txt'):
    # Use pinned wheels for Python 3.10
    sh('pip install -r requirements.txt')
else:
    # Fallback for Python 3.12+: use ONNX path only
    sh('pip install --index-url https://download.pytorch.org/whl/cpu torch==2.5.1+cpu torchaudio==2.5.1+cpu')
    sh('pip install sherpa-onnx fastapi uvicorn[standard] python-multipart requests sentencepiece numpy==1.26.4')
    # Patch model.py to be resilient when k2/sherpa are unavailable
    mp = pathlib.Path('model.py')
    s = mp.read_text(encoding='utf-8')
    lines = s.splitlines(True)  # keep newlines
    # Ensure future annotations at top
    if not any(line.startswith('from __future__ import annotations') for line in lines[:2]):
        lines.insert(0, 'from __future__ import annotations\n')
    # Replace 'import k2  # noqa' with safe fallback
    def _replace_line(lines, predicate, block_lines):
        for idx, ln in enumerate(lines):
            if predicate(ln):
                lines[idx:idx+1] = block_lines
                return True
        return False
    block_k2 = [
        'try:\n',
        '    import k2  # noqa\n',
        'except Exception:\n',
        '    k2 = None\n',
    ]
    block_sherpa = [
        'try:\n',
        '    import sherpa as _sherpa\n',
        '    sherpa = _sherpa\n',
        'except Exception:\n',
        '    class _DummyBase: ...\n',
        '    class _SherpaModule:\n',
        '        OfflineRecognizer = _DummyBase\n',
        '        OnlineRecognizer = _DummyBase\n',
        '    sherpa = _SherpaModule()\n',
    ]
    _replace_line(lines, lambda ln: ln.strip() == 'import k2  # noqa', block_k2)
    _replace_line(lines, lambda ln: ln.strip() == 'import sherpa', block_sherpa)
    mp.write_text(''.join(lines), encoding='utf-8')

    # Ensure 'import sherpa_onnx' exists and remove any stray '_onnx' tokens from prior edits
    if not any(ln.strip().startswith('import sherpa_onnx') for ln in lines):
        # insert after the sherpa block or at top after future import
        ins = 1 if lines and lines[0].startswith('from __future__ import annotations') else 0
        lines.insert(ins, 'import sherpa_onnx\n')
    # purge orphan _onnx tokens
    lines[:] = [ln for ln in lines if ln.strip() != '_onnx']
    mp.write_text(''.join(lines), encoding='utf-8')
    # Fix any stray 'try:' lines without an indented body (from previous runs)
    fixed = []
    i = 0
    while i < len(lines):
        ln = lines[i]
        if ln.strip() == 'try:' and (i+1 >= len(lines) or (not (lines[i+1].startswith(' ') or lines[i+1].startswith('	')))):
            # drop this stray try
            i += 1
            continue
        fixed.append(ln)
        i += 1
    lines = fixed
    mp.write_text(''.join(lines), encoding='utf-8')
# Ensure ngrok (for public URL tunnel)
sh('pip install -q pyngrok')

print('Environment ready')


In [ ]:
# 2) Verify files
import os, glob, platform, sys
print('Platform:', platform.platform())
print('CWD:', os.getcwd())
print('Has model.py?', os.path.exists('model.py'))
print('Found ONNX encoder?', os.path.exists('encoder-epoch-20-avg-10.int8.onnx') or os.path.exists('encoder-epoch-20-avg-10.onnx'))
print('Found ONNX decoder?', os.path.exists('decoder-epoch-20-avg-10.int8.onnx') or os.path.exists('decoder-epoch-20-avg-10.onnx'))
print('Found ONNX joiner?', os.path.exists('joiner-epoch-20-avg-10.int8.onnx') or os.path.exists('joiner-epoch-20-avg-10.onnx'))
print('Found config.json?', os.path.exists('config.json'))
print('Vietnamese test wavs:', glob.glob('test_wavs/vietnamese/*.wav'))


In [ ]:
# 3) Quick local inference
from model import get_pretrained_model, decode
REPO_ID = 'hynt/sherpa-onnx-zipformer-vi-int8-2025-10-16'
DECODING_METHOD = 'modified_beam_search'
NUM_ACTIVE_PATHS = 15
TEST_WAV = 'test_wavs/vietnamese/0.wav'
rec = get_pretrained_model(REPO_ID, decoding_method=DECODING_METHOD, num_active_paths=NUM_ACTIVE_PATHS)
print('Recognizer ready')
print('Recognized text:', decode(rec, TEST_WAV))


## Public API server
- `GET /healthz`
- `GET /readyz`
- `POST /v1/transcribe` (multipart key `file`, or JSON with `audio_url`/`audio_base64`)
- Set `REQUIRE_API_KEY=true` and `API_KEY=...` to enforce auth


In [ ]:
#@title 4) Start FastAPI (background)
import os, threading, time, requests, uvicorn
os.environ.setdefault('UVICORN_PORT', '8000')
os.environ.setdefault('REQUIRE_API_KEY', 'false')
os.environ.setdefault('API_KEY', '')
os.environ.setdefault('MODEL_REPO_ID', 'hynt/sherpa-onnx-zipformer-vi-int8-2025-10-16')
os.environ.setdefault('DECODING_METHOD', 'modified_beam_search')
os.environ.setdefault('NUM_ACTIVE_PATHS', '15')
os.environ.setdefault('MAX_DURATION_SEC', '60')
def _run():
    uvicorn.run('api_server:app', host='0.0.0.0', port=int(os.environ['UVICORN_PORT']), workers=1)
thr = threading.Thread(target=_run, daemon=True)
thr.start()
base = f"http://127.0.0.1:{os.environ['UVICORN_PORT']}"
for _ in range(60):
    try:
        r = requests.get(base + '/healthz', timeout=1)
        if r.ok:
            print('✅ Local server:', base)
            break
    except Exception:
        time.sleep(0.2)


In [ ]:
#@title 5) Setup ngrok auth (optional)
from pyngrok import ngrok, conf
import getpass
print('To use a public URL, set your ngrok authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
ngrok_token = getpass.getpass('Enter ngrok authtoken (or press Enter to skip): ')
if ngrok_token:
    conf.get_default().auth_token = ngrok_token
    print('ngrok token configured')
else:
    print('Proceeding without token; tunnel may be limited')


In [ ]:
#@title 6) Expose a public URL via ngrok
from pyngrok import ngrok
import requests, os, time
port = os.environ.get('UVICORN_PORT', '8000')
public_url = None
# Close any existing tunnels
try:
    ngrok.kill()
except Exception:
    pass
# Create a new tunnel
try:
    tunnel = ngrok.connect(addr=int(port))
    public_url = str(tunnel.public_url)
    print('Public API:', public_url)
    # Health checks
    print('Health:', requests.get(public_url + '/healthz', timeout=10).json())
    print('Ready :', requests.get(public_url + '/readyz', timeout=10).json())
except Exception as e:
    print('Failed to create ngrok tunnel:', e)
    raise


In [ ]:
# 7) Call the API (examples)
import requests, base64, os
API_BASE = public_url
API_KEY = os.environ.get('API_KEY', '')
use_auth = os.environ.get('REQUIRE_API_KEY','false').lower()=='true'
headers = ({'Authorization': f'Bearer {API_KEY}'} if use_auth else {})
# Base64 JSON only (more reliable through tunnels)
with open('test_wavs/vietnamese/1.wav', 'rb') as f:
    b64 = base64.b64encode(f.read()).decode('utf-8')
payload = {'audio_base64': b64, 'decoding_method': 'modified_beam_search', 'num_active_paths': 15}
r = requests.post(API_BASE + '/v1/transcribe', json=payload, headers=headers, timeout=60)
print('Base64  :', r.status_code, (r.text if not r.ok else r.json()))
